# <b>Task03 - Traffic Category Classification</b>

___
## Table of Content

* Load raw features
* Train the multi-modal autoencoder
* Train the raw features models
* Validate the models
* k-nearest-neighbors class probability

In [1]:
# type: ignore

from mltoolbox.classification import DeepClassifier, KnnClassifier
from mltoolbox.representation import MultimodalAE
from sklearn.metrics import classification_report
from mltoolbox.metrics import k_class_proba_report
from src.utils import get_datasets
from keras import layers
import pandas as pd
import numpy as np
import joblib

2022-12-07 22:28:59.403920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 22:28:59.592597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda3/envs/bigdatalab/lib/libfabric:/usr/local/cuda/lib64
2022-12-07 22:28:59.592628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 22:28:59.637215: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already

In [2]:
task = 'task03' # Define the task. This variable is used mainly for savings
K = 0 # Specify the considered fold (Stratified k fold validation)

## Load raw features



In [11]:
# Load ports word2vec embeddings - entity
payload=pd.read_csv(f'../data/{task}/features/payload.csv', index_col=[0])
# Load statistics features - quantity
statistics=pd.read_csv(f'../data/{task}/features/statistics.csv', index_col=[0])
# Load statistics sequences - quantity
sequences=pd.read_csv(f'../data/{task}/features/sequences.csv', index_col=[0])
# Load ip address word2vec embeddings - entity
ipaddress=pd.read_csv(f'../data/{task}/features/ipaddress.csv', index_col=[0])
# Merge the features as raw concatenation
concat = payload.reset_index().drop(columns=['label'])\
     .merge(statistics.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(sequences.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(ipaddress.reset_index(), on='index', how='inner')\
     .set_index('index')
# Load stratified k folds
kfolds = joblib.load(f'../data/{task}/skfolds/folds.save')

## Train the multi-modal autoencoder

In [14]:
feature, fname = concat, 'mae'
# Get the features size. Last column is the label one
P,St,Se,I = payload.shape[1]-1, statistics.shape[1]-1, \
            sequences.shape[1]-1, ipaddress.shape[1]-1

# Retrieve the training and validation samples from the k-folds order
X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)

In [15]:
P, St, Se, I

(32, 72, 128, 64)

In [16]:
# Define the classifier architecture
inputs = layers.Input(X_train.shape[1],)

# Encoder branch of modality 1 - payload embeddings
hidden1 = layers.Lambda(lambda x: x[:, :P])(inputs)
hidden1 = layers.Reshape((-1, 1))(hidden1)
hidden1 = layers.Dense(64, activation='relu')(hidden1)
hidden1 = layers.Conv1D(32, 3, activation='relu')(hidden1)
hidden1 = layers.Dropout(.3)(hidden1)
hidden1 = layers.MaxPooling1D(2)(hidden1)
hidden1 = layers.Dropout(.3)(hidden1)
hidden1 = layers.Flatten()(hidden1)
hidden1 = layers.Dense(32, activation='relu')(hidden1)

# Encoder branch of modality 2 - statistics
hidden2 = layers.Lambda(lambda x: x[:, P:St])(inputs)
hidden2 = layers.Dense(32, activation='relu')(hidden2)

# Encoder branch of modality 3 - sequences
hidden3 = layers.Lambda(lambda x: x[:, P+St:P+St+Se])(inputs)
hidden3 = layers.Reshape((32, 4))(hidden3)
hidden3 = layers.Conv1D(32, 3, activation='relu')(hidden3)
hidden3 = layers.Dropout(.3)(hidden3)
hidden3 = layers.MaxPooling1D(2)(hidden3)
hidden3 = layers.Dropout(.3)(hidden3)
hidden3 = layers.Flatten()(hidden3)
hidden3 = layers.Dense(32, activation='relu')(hidden3)

# Encoder branch of modality 4 - ip address embeddings
hidden4 = layers.Lambda(lambda x: x[:, P+St+Se:P+St+Se+I])(inputs)
hidden4 = layers.Dense(32, activation='relu')(hidden4)

# Concatenate
hidden = layers.Concatenate()([hidden1, hidden2, hidden3, hidden4])
# Common encoder
hidden = layers.Dense(512, activation='relu')(hidden)
hidden = layers.Dense(256, activation='relu')(hidden)
# Bottleneck
hidden = layers.Dense(64, activation='relu', name='Coded')(hidden)
# Common decoder
hidden = layers.Dense(256, activation='relu')(hidden)
hidden = layers.Dense(512, activation='relu')(hidden)
hidden = layers.Dense(32*4, activation='relu')(hidden)

hidden1 = layers.Dense(32, activation='relu')(hidden)
hidden1 = layers.Dense(480, activation='relu')(hidden)
hidden1 = layers.Reshape((15, 32))(hidden1)
hidden1 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden1)
hidden1 = layers.UpSampling1D(2)(hidden1)
hidden1 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden1)
hidden1 = layers.UpSampling1D(2)(hidden1)
hidden1 = layers.Flatten()(hidden1)
output1 = layers.Dense(P, activation='linear', name='payload')(hidden1)

hidden2 = layers.Dense(32, activation='relu')(hidden)
output2 = layers.Dense(St, activation='linear', name='statistics')(hidden2)

hidden3 = layers.Dense(32, activation='relu')(hidden)
hidden3 = layers.Dense(480, activation='relu')(hidden3)
hidden3 = layers.Reshape((15, 32))(hidden3)
hidden3 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden3)
hidden3 = layers.UpSampling1D(2)(hidden3)
hidden3 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden3)
hidden3 = layers.UpSampling1D(2)(hidden3)
hidden3 = layers.Flatten()(hidden3)
output3 = layers.Dense(Se, activation='linear', name='sequences')(hidden3)

hidden4 = layers.Dense(32, activation='relu')(hidden)
output4 = layers.Dense(I, activation='linear', name='ipaddress')(hidden4)

outputs = [output1, output2, output3, output4]

# Mean Squared Errors
loss = {'payload':'mse', 'statistics':'mse', 'sequences':'mse', 'ipaddress':'mse'} 
weights = {'payload':1/P, 'statistics':1/St, 'sequences':1/Se, 'ipaddress':1/I} # Balance losses

2022-12-07 22:54:50.818867: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda3/envs/bigdatalab/lib/libfabric:/usr/local/cuda/lib64
2022-12-07 22:54:50.818930: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-07 22:54:50.818962: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vm5-bigdata): /proc/driver/nvidia/version does not exist
2022-12-07 22:54:50.819363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Initialize the classifier
mae = MultimodalAE(model_path=f'../data/{task}/mae/{fname}_k{K}',
                   io=(inputs, outputs), losses=loss, weights=weights)
# Fit the multi-modal autoencoder
mae.fit(training_data=(X_train, X_train), y_sizes=[P, St, Se, I], batch_size=256, 
        scale_data=True, epochs=3, validation_data=(X_val, X_val), save=True, 
        verbose=0)
embeddings = np.vstack([mae.transform(X_train), mae.transform(X_val)])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: ../data/task03/mae/mae_k0_mae/assets


INFO:tensorflow:Assets written to: ../data/task03/mae/mae_k0_mae/assets


In [19]:
# Save the embeddings
embeddings = pd.DataFrame(embeddings, index=np.hstack(kfolds[K][:2]))
embeddings[['label']] = np.hstack(kfolds[K][2:])
embeddings.to_csv(f'../data/{task}/embeddings/mae_embeddings_k{K}.csv')

## Train the raw-features-models

In [21]:
embeddings=pd.read_csv(f'../data/{task}/embeddings/mae_embeddings_k{K}.csv', 
                       index_col=[0])
n_classes = ipaddress.value_counts('label').shape[0]

In [23]:
features = [payload, statistics, sequences, ipaddress, concat, embeddings]
feature_names = ['payload', 'statistics', 'sequences', 'ipaddress', 'rawcat', 'mae']

for feature, fname in zip(features, feature_names):
    print(f'\nTraining {fname} deep classifier')
    
    # Retrieve the training and validation samples from the k-folds order
    X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
    # Define the classifier architecture
    inputs = layers.Input(X_train.shape[1],)
    hidden = layers.Dense(512, activation='relu')(inputs)
    hidden = layers.Dropout(.3)(hidden)
    hidden = layers.Dense(256, activation='relu')(hidden)
    hidden = layers.Dropout(.3)(hidden)
    outputs = layers.Dense(n_classes, activation='softmax')(hidden)
    # Initialize the classifier
    mpath = f'../data/{task}/classifiers/{fname}_k{K}'
    classifier = DeepClassifier(io=(inputs, outputs), model_path=mpath)
    # Train the classifier for 3 epochs. Standardize data before training
    classifier.fit(training_data=(X_train, y_train), 
                   validation_data=(X_val, y_val), 
                   scale_data=True, batch_size=256, epochs=3, save=True)


Training payload deep classifier
Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/2 [==============>...............] - ETA: 0s - loss: 0.6990 - accuracy: 0.2070WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f478412bcb0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: ../data/task03/classifiers/mae_k0_classifier/assets


INFO:tensorflow:Assets written to: ../data/task03/classifiers/mae_k0_classifier/assets


2/2 [==============================] - 2s 1s/step - loss: 0.7046 - accuracy: 0.2983 - val_loss: 1.6192 - val_accuracy: 0.3000
Epoch 2/3
2/2 [==============================] - 0s 51ms/step - loss: 0.5585 - accuracy: 0.3031 - val_loss: 1.7649 - val_accuracy: 0.1526
Epoch 3/3
2/2 [==============================] - 0s 49ms/step - loss: 0.5096 - accuracy: 0.3604 - val_loss: 1.8581 - val_accuracy: 0.2105


## Validate the models

In [24]:
features = [payload, statistics, sequences, ipaddress, concat, embeddings]
feature_names = ['payload', 'statistics', 'sequences', 'ipaddress', 'rawcat', 'mae']

for feature, fname in zip(features, feature_names):
    # Retrieve the training and validation samples from the k-folds order
    X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
    mpath = f'../data/{task}/classifiers/{fname}_k{K}'
    classifier = DeepClassifier(_load_model=True, model_path=mpath)
    y_pred = classifier.predict(X_val, scale_data=True)
    report = classification_report(y_val, y_pred, labels=np.unique(y_val), 
                                   output_dict=True)

    f1 = report['macro avg']['f1-score']
    
    pd.DataFrame(report).T.to_csv(f'../data/interim/{fname}_deep_k{K}.csv')
    print(f'\nValidating {fname} deep classifier:\n\tMacro avg f1:{f1}')

/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Validating payload deep classifier:
	Macro avg f1:0.2714312691878126

Validating statistics deep classifier:
	Macro avg f1:0.41261250941908056


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Validating sequences deep classifier:
	Macro avg f1:0.14400450189923875


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier


Validating ipaddress deep classifier:
	Macro avg f1:0.19481827911954955

Validating rawcat deep classifier:
	Macro avg f1:0.45983898848902516

Validating mae deep classifier:
	Macro avg f1:0.28569060930565576


## k-nearest-neighbors class probability</b>

In [26]:
features = [concat, embeddings]
feature_names = ['rawcat', 'mae']
for feature, fname in zip(features, feature_names):
    for k in range(1, 20):
        # Retrieve the training and validation samples from the k-folds order
        X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
        X, y = np.vstack([X_train, X_val]), np.hstack([y_train, y_val])
        
        knn = KnnClassifier(n_neighbors=k, metric='cosine')
        knn.fit(X, y, scale_data=True)

        to_keep = np.where(y!='unknown')[0].reshape(-1, 1)
        pcs = knn.predict_proba(to_keep)
        
        y_true = y[np.ravel(to_keep)]
        report = k_class_proba_report(y_true, pcs, output_dict=True)
        
        kpc = report['macro avg']['kpc']

        pd.DataFrame(report).T.to_csv(f'../data/interim/{fname}_{k}pc_k{K}.csv')
        print(f'\nMacro avg {k}Pc - {fname}: {kpc}')

/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Macro avg 1Pc - rawcat: 0.69


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Macro avg 2Pc - rawcat: 0.63


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


...


Macro avg 17Pc - mae: 0.5


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Macro avg 18Pc - mae: 0.49


/opt/anaconda3/envs/bigdatalab_cpu_202208/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Macro avg 19Pc - mae: 0.49
